In [227]:
%run imports.ipynb
%run '08222019_functions.ipynb'

In [228]:
calldata_biallel= zarr.open('data/phase2_biallel_allele_count.zarr/')
out_alleles = h5py.File('data/outgroup_alleles_phase2.h5')
annotation = zarr.open('data/ag1000g.phase2.ar1.pass.biallelic_snpeff/')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


In [229]:
pos = allel.SortedIndex(callset_biallel['3R']["variants/POS"])

In [230]:
def get_neutral(chrom, pos):
    if chrom == '3R':
        cond = (pos > 1000000) & (pos < 37000000)
    return cond

In [231]:
cond_neut = get_neutral(chrom, pos)
cond_neut

array([False, False, False, ..., False, False, False])

In [232]:
ann = annotation['3R']['variants/ANN_Annotation'][:]

In [235]:
cond_inter = ann == 'intergenic_region'
cond_inter

array([False, False, False, ...,  True,  True,  True])

In [237]:
pos_neu_inter = pos[cond_neut & cond_inter] 
pos_neu_inter

<SortedIndex shape=(3651719,) dtype=int32>
[1000005, 1000007, 1000012, 1000013, 1000014, ..., 36935274, 36935277, 36935282, 36935283, 36935284]

In [238]:
genotypes = allel.GenotypeChunkedArray(callset_biallel['3R']["calldata/GT"])
genotypes

<GenotypeChunkedArray shape=(10752701, 1142, 2) dtype=int8 chunks=(524288, 61, 2)
   nbytes=22.9G cbytes=-1 cratio=-24559169084.0
   compression=blosc compression_opts={'cname': 'zstd', 'clevel': 1, 'shuffle': -1, 'blocksize': 0}
   values=zarr.core.Array>

--------------
Subset the metadata to my 3 pops of interest

In [239]:
populations = 'KE', 'GM', 'GW'

In [240]:
sample_selection = metadata.population.isin({'KE', 'GM', 'GW'}).values

In [241]:
samples_subset = metadata[sample_selection]
samples_subset.reset_index(drop=True, inplace=True)
samples_subset.head()

ox_code src_code population      country    location            site  \
0  AG0082-C     2518         GM  Gambia, The  Njabakunda  Sare Illo Buya   
1  AG0085-C     2521         GM  Gambia, The  Njabakunda  Sare Illo Buya   
2  AG0089-C     2525         GM  Gambia, The  Njabakunda  Sare Illo Buya   
3  AG0096-C     2532         GM  Gambia, The  Njabakunda  Sare Illo Buya   
4  AG0097-C     2533         GM  Gambia, The  Njabakunda  Sare Illo Buya   

       contributor          contact  year  m_s sex  n_sequences  \
0  Martin Donnelly  Martin Donnelly  2011  NaN   F     93088052   
1  Martin Donnelly  Martin Donnelly  2011  NaN   F    103517758   
2  Martin Donnelly  Martin Donnelly  2011  NaN   F    114898838   
3  Martin Donnelly  Martin Donnelly  2011  NaN   F    105627176   
4  Martin Donnelly  Martin Donnelly  2011  NaN   F     95764982   

   mean_coverage ebi_sample_acc  latitude  longitude  
0          30.53      ERS248676     13.55      -15.9  
1          32.14      ERS248679     13.55      -15.9  
2          35.13      ERS248683     13.55      -15.9  
3          32.73      ERS248690     13.55      -15.9  
4          23.23      ERS248691     13.55      -15.9

In [242]:
samples_subset.population.value_counts()

GW    91
GM    65
KE    48
Name: population, dtype: int64

----------
Subset the genotype to variants passed and pops

In [245]:
%%time
genotypes_subset = genotypes.subset(sel1 = sample_selection)

CPU times: user 1min 45s, sys: 24.8 s, total: 2min 10s
Wall time: 1min 38s


In [246]:
genotypes_subset

<GenotypeChunkedArray shape=(10752701, 204, 2) dtype=int8 chunks=(10501, 204, 2)
   nbytes=4.1G cbytes=171.6M cratio=24.4
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

-----------
Counting the alleles:

In [247]:
subpops = {
    'all': list(range(len(samples_subset))),
    'GM': samples_subset[samples_subset.population == 'GM'].index.tolist(),
    'GW': samples_subset[samples_subset.population == 'GW'].index.tolist(),
    'KE': samples_subset[samples_subset.population == 'KE'].index.tolist(),

}

In [275]:
allele_counts = genotypes_subset.count_alleles_subpops(subpops, max_allele=3)
allele_counts

<AlleleCountsChunkedTable shape=(10752701,) dtype=[('GM', '<i4', (4,)), ('GW', '<i4', (4,)), ('KE', '<i4', (4,)), ('all', '<i4', (4,))]
   nbytes=656.3M cbytes=51.9M cratio=12.6
   values=allel.chunked.storage_zarr.ZarrTable>

In [277]:
ac = {}
for pop in list(subpops):
        ac[pop] = allel.AlleleCountsChunkedArray(allele_counts[pop][:][cond_neut & cond_inter])

In [279]:
bool_bial = ac['all'].allelism() == 2
bool_bial

<ChunkedArrayWrapper shape=(3651719,) dtype=bool chunks=(456465,)
   nbytes=3.5M cbytes=2.0M cratio=1.7
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

In [280]:
for pop in list(subpops):
        ac[pop] = ac[pop][bool_bial]

In [281]:
pos_final = pos_neu_inter[bool_bial]
pos_final

array([ 1000005,  1000007,  1000027, ..., 36935273, 36935277, 36935283],
      dtype=int32)

In [282]:
for chrom in contigs:
    print(chrom, pos.shape)
    print(chrom, pos_final.shape)
    print(chrom, ac['all'].shape)
    print(chrom, ac['KE'].shape)

3R (10752701,)
3R (1329636,)
3R (1329636, 4)
3R (1329636, 4)


In [283]:
#out_alleles = zarr.open('data/outgroup_alleles_phase2.zarr/')

In [284]:
def consensus_aa(chrom, out_alleles, outgroups):
    aa = None
    for out in outgroups:
        if aa is None:
            aa = out_alleles[chrom][out]
        else:
            loc_consensus = aa == out_alleles[chrom][out]
            aa[~loc_consensus] = b'.'
    return aa

In [287]:
cons_4 = consensus_aa('3R', out_alleles, ['chri'])
cons_4 

<HDF5 dataset "chri": shape (10752701,), type "|S1">

In [288]:
cons_4_filt = cons_4[cond_neut & cond_inter]
cons_4_filt_final = cons_4_filt.compress(bool_bial)
cons_4_filt_final

array([b'A', b'A', b'T', ..., b'.', b'.', b'.'], dtype='|S1')

In [289]:
ref = callset_biallel['3R']['variants/REF'][:] 
ref_filt = ref[cond_neut & cond_inter]
ref_filt_final = ref_filt[bool_bial]

In [290]:
alt =  callset_biallel['3R']['variants/ALT']
alt_filt = np.compress((cond_neut & cond_inter), alt, axis=0) 
alt_filt_final = np.compress(bool_bial, alt_filt, axis=0) 

In [291]:
ref_filt_final.shape, alt_filt_final.shape

((1329636,), (1329636,))

In [292]:
loc_bi_seg = (ac['all'].allelism() == 2) & (ac['all'][:, 0] > 0) & (ac['all'][:, 1] > 0)
log(nnz(loc_bi_seg))

# locate positions where aa is determined and matches Ag1000G
loc_aa_match = (cons_4_filt_final!= b'.') & (cons_4_filt_final == ref_filt_final) | (cons_4_filt_final == alt_filt_final)
log(nnz(loc_aa_match))

# locate positions with missing calls
an = ac['all'].sum(axis=1)
loc_nomiss = an == an.max() 
log(nnz(loc_nomiss))

# locations to use


1329636
851575
1314146


In [295]:
loc_asc = loc_bi_seg & loc_aa_match & loc_nomiss
loc_asc 

array([False,  True,  True, ..., False, False, False])

In [297]:
for chrom in contigs:
    print(loc_asc.sum())

843323


In [298]:
for chrom in contigs:
    print(cons_4_filt_final.shape) 

(1329636,)


In [299]:
for chrom in contigs:
    print(cons_4_filt_final.shape)

(1329636,)


In [303]:
# kenya
samples_fn = 'samples.meta.txt'
samples = etl.fromtsv(samples_fn).selecteq('population', 'KE')

In [308]:
kenya_ind = samples.select(lambda r: r.location == 'Kilifi').values('ox_code').list()

geno_filt = allel.GenotypeChunkedArray(callset_biallel['3R']["calldata/GT"]).compress(cond_neut & cond_inter)
#geno_filt = {chrom: np.compress((cond_neut[chrom] & cond_inter[chrom]), geno[chrom], axis=0) for chrom in contigs}

geno_filt_final = np.compress(bool_bial, geno_filt, axis=0) 
del geno_filt

# take just the kenya genotypes

In [309]:
import anhima

In [316]:
sample_selection = metadata.population.isin({'KE'}).values

In [317]:
g = geno_filt_final.subset(sel1=sample_selection)

In [319]:
ac['KE_sav'] =  g.count_alleles()

In [320]:
del geno_filt_final
del g

In [322]:
populations1 = ['KE']
populations2 = ['GM', 'GW']

In [327]:
for pop1, pop2 in zip(populations1, populations2):
    for chrom in contigs:
        with open('dadi/%s_%s_4out_%s_dec.txt' % (pop1, pop2, chrom), 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=' ')
            writer.writerow(['ingroup', 'out', 'Allele1', pop1, pop2, 'Allele2', pop1, pop2, 'gene', 'Position'])
            for ingr, out, ac_pop1, ac_pop2, allele2, position in zip(ref_filt_final[loc_asc], cons_4_filt_final[loc_asc], ac[pop1][loc_asc], ac[pop2][loc_asc], alt_filt_final[loc_asc], pos_final[loc_asc]):
                ingr = ingr.decode()
                out = out.decode()
                allele2 = allele2[0]
                writer.writerow(['-' + ingr + '-', '-' + out + '-', ingr, ac_pop1[0], ac_pop2[0], allele2, ac_pop1[1], ac_pop2[1], '---', position] )